In [392]:
list_df = pd.DataFrame(index=listed)
fla = []
valu = []
for code_start in listed:
    r = r + 1
    path = os.path.join(my_path, "../data/stock/")
    df_com = pd.read_csv(path + code_start + "_tick_df.csv")
    my_path = os.path.abspath(os.path.dirname('__file__'))
    path = os.path.join(my_path, "../data/cpickle/")
    art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
    art_ratios = art_ratios["mv"][code_start]
    art_ratios["close"] = (art_ratios["MV_pred"] / art_ratios["MV_pred"].iloc[0]) * 100
    art_ratios["close"] = (art_ratios["close"] + df_com["close"]) / 2
    rela_flo = (art_ratios["close"].iloc[-1] / df_com["close"].iloc[-1]) - 1
    if (rela_flo > -0.05) and (rela_flo < 0.05):
        valu.append("Fair Valued")
    elif (rela_flo < 0.05):
        valu.append("Overvalued")
    else:
        valu.append("Undervalued")
    fla.append(round(rela_flo * 100, 2))

list_df["Value"] = valu
list_df["Extent"] = fla

In [426]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(3.4, 4.4))

scaler.fit(list_df["Extent"].values.reshape(-1, 1))

list_df["Rating"] = scaler.transform(list_df["Extent"].values.reshape(-1, 1))

list_df["Rating"] = list_df["Rating"] * (1 + (list_df["Extent"].mean() / 100))

list_df["Rating"] = list_df["Rating"].round(2)


In [427]:
list_df

,Value,Extent,Rating
code_or_ticker,,,
CAKE,Overvalued,-17.67,3.38
RRGB,Overvalued,-22.47,3.29
BJRI,Undervalued,30.48,4.26


In [428]:
unlist_df = pd.DataFrame(index=codes)
fla = []
valu = []
for code_start in codes:
    r = r + 1
    my_path = os.path.abspath(os.path.dirname('__file__'))
    path = os.path.join(my_path, "../data/cpickle/")
    art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
    art_ratios = art_ratios["mv"][code_start]
    art_ratios["close"] = (art_ratios["MV_pred"] / art_ratios["MV_pred"].iloc[0]) * 100
    if code_start in list(listed):
        print("go")
        path = os.path.join(my_path, "../data/stock/")
        art_ratios = pd.read_csv(path + code_start + "_tick_df.csv")

    rela_flo = (art_ratios["close"].iloc[-1] / df2["mean"].iloc[-1]) - 1
    if (rela_flo > -0.05) and (rela_flo < 0.05):
        valu.append("Neutral")
    elif (rela_flo < 0.05):
        valu.append("Under")
    else:
        valu.append("Over")
    fla.append(round(rela_flo * 100, 2))

unlist_df["Value"] = valu
unlist_df["Extent"] = fla

go
go
go


In [390]:
list_df

NameError: name 'list_df' is not defined

In [374]:
unlist_df

,Value,Extent
code_or_ticker,,
APPB,Neutral,4.68
CHIL,Over,16.38
CPKI,Under,-29.01
TGIF,Under,-16.92
CAKE,Over,24.82
RRGB,Over,44.81
BJRI,Under,-33.92


In [429]:
from datetime import timedelta

from sklearn.preprocessing import MinMaxScaler

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../input_fields.csv")

input_fields = pd.read_csv(path)

not_listed = input_fields[input_fields["listed"] == "No"].reset_index(drop=True)["code_or_ticker"]
path = os.path.join(my_path, "../data/stock/")

df2 = pd.read_csv(path+"bench_tick_df.csv")
df2["date"] = pd.to_datetime(df2["date"], format="%Y-%m-%d")
## Loop over all
codes = input_fields["code_or_ticker"]
listed = input_fields[input_fields["listed"] == "Yes"]["code_or_ticker"]
big_avg = pd.DataFrame()
r = -1

fla = []
valu = []
# listed = ["BJRI"]
dict_list = {}
for code_start in listed:
    r = r + 1
    path = os.path.join(my_path, "../data/stock/")
    df_com = pd.read_csv(path + code_start + "_tick_df.csv")
    df_com["date"] = pd.to_datetime(df_com["date"], format="%Y-%m-%d")
    my_path = os.path.abspath(os.path.dirname('__file__'))
    path = os.path.join(my_path, "../data/cpickle/")
    art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
    art_ratios = art_ratios["mv"][code_start]
    art_ratios["close_sim"] = (art_ratios["MV_pred"] / art_ratios["MV_pred"].iloc[0]) * 100
    art_ratios["date"] = pd.to_datetime(art_ratios["date"], format="%Y-%m-%d")
    art_ratios = pd.merge(art_ratios, df_com, on="date", how="left")
    art_ratios["close_sim"] = (art_ratios["close_sim"] + art_ratios["close"]) / 2
    art_ratios["Extend"] = (art_ratios["close_sim"] / art_ratios["close"]) - 1
    art_ratios = art_ratios[art_ratios["date"] > art_ratios["date"].max() - timedelta(365 * 4)]

    scaler = MinMaxScaler(feature_range=(2.5, 3))
    art_ratios = art_ratios.fillna(method="bfill").fillna(method="ffill")
    scaler.fit(art_ratios["Extend"].values.reshape(-1, 1))

    art_ratios["Rating List"] = scaler.transform(art_ratios["Extend"].values.reshape(-1, 1))
    art_ratios["Rating List"] = art_ratios["Rating List"]*(list_df.loc[code_start,'Rating']/art_ratios["Rating List"].iloc[-1])
    new_frame_listed = art_ratios[["date", "Rating List"]]
    dict_list[code_start] = new_frame_listed

    
    

In [452]:
df_com = pd.read_csv(path + code_start + "_tick_df.csv")

In [451]:
new_frame_listed.tail()

,date,Rating List
1514,2018-01-09,4.253980
1515,2018-01-10,4.260784
1516,2018-01-11,4.258041
1517,2018-01-12,4.255903
1518,2018-01-16,4.260000


In [430]:
list_df

,Value,Extent,Rating
code_or_ticker,,,
CAKE,Overvalued,-17.67,3.38
RRGB,Overvalued,-22.47,3.29
BJRI,Undervalued,30.48,4.26


In [406]:
art_ratios["Rating List"] = art_ratios["Rating List"]*(list_df.loc[code,'Rating']/art_ratios["Rating List"].iloc[-1])

In [402]:
code="BJRI"
list_df.loc[code,'Rating']

4.26

In [431]:
dict_list["BJRI"].max()

date           2018-01-16 00:00:00
Rating List                4.79105
dtype: object

In [433]:
dict_list["CAKE"].max()

date           2018-01-16 00:00:00
Rating List                3.71263
dtype: object

In [450]:
unlist_df

,Value,Extent,Rating
code_or_ticker,,,
APPB,Neutral,4.68,3.69
CHIL,Over,16.38,3.97
CPKI,Under,-29.01,2.86
TGIF,Under,-16.92,3.16
CAKE,Over,24.82,4.18
RRGB,Over,44.81,4.67
BJRI,Under,-33.92,2.74


In [447]:
scaler = MinMaxScaler(feature_range=(2.7, 4.6))

scaler.fit(unlist_df["Extent"].values.reshape(-1, 1))

unlist_df["Rating"] = scaler.transform(unlist_df["Extent"].values.reshape(-1, 1))

unlist_df["Rating"] = unlist_df["Rating"] * (1 + (unlist_df["Extent"].mean() / 100))
unlist_df["Rating"] = unlist_df["Rating"].round(2)

In [448]:
from sklearn.preprocessing import MinMaxScaler

dict_unlist = {}
for code_start in codes:
    r = r + 1
    my_path = os.path.abspath(os.path.dirname('__file__'))
    path = os.path.join(my_path, "../data/cpickle/")
    part_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
    part_ratios = part_ratios["mv"][code_start]
    part_ratios["close_sim"] = (part_ratios["MV_pred"] / part_ratios["MV_pred"].iloc[0]) * 100
    part_ratios["date"] = pd.to_datetime(part_ratios["date"], format="%Y-%m-%d")
    if code_start in list(listed):
        print("go")
        path = os.path.join(my_path, "../data/stock/")
        part_ratios = pd.read_csv(path + code_start + "_tick_df.csv")
        part_ratios["date"] = pd.to_datetime(part_ratios["date"], format="%Y-%m-%d")
        part_ratios["close_sim"] = part_ratios["close"]
    part_ratios = pd.merge(part_ratios, df2, on="date", how="left")
    part_ratios["Extend"] = (part_ratios["close_sim"] / part_ratios["mean"]) - 1
    part_ratios = part_ratios[part_ratios["date"] > part_ratios["date"].max() - timedelta(365 * 4)]

    scaler = MinMaxScaler(feature_range=(2.5, 3))
    part_ratios = part_ratios.fillna(method="bfill").fillna(method="ffill")
    scaler.fit(part_ratios["Extend"].values.reshape(-1, 1))

    part_ratios["Rating Unlist"] = scaler.transform(part_ratios["Extend"].values.reshape(-1, 1))
    part_ratios["Rating Unlist"] = part_ratios["Rating Unlist"]*(unlist_df.loc[code_start,'Rating']/part_ratios["Rating Unlist"].iloc[-1])
    
    

    new_frame_unlisted = part_ratios[["date", "Rating Unlist"]]
    dict_unlist[code_start] = new_frame_unlisted


go
go
go


In [449]:
dict_unlist["CAKE"].max()

date             2018-01-18 00:00:00
Rating Unlist                4.55247
dtype: object

In [441]:
fa = -1
frame_new = pd.DataFrame()
for code_start in codes:
    fa = fa + 1
    if code_start in list(listed):
        new_frame_unlisted = pd.merge(dict_unlist[code_start], dict_list[code_start], on="date", how="left")
        #### This multiplier should be placed to list_df
        new_frame_unlisted["Rating Unlist"] = new_frame_unlisted["Rating Unlist"] * (
        1 + (unlist_df["Extent"].mean() / 100))
        new_frame_unlisted["Rating List"] = new_frame_unlisted["Rating List"] * (1 + (unlist_df["Extent"].mean() / 100))
        new_frame_unlisted[code_start] = new_frame_unlisted[["Rating Unlist", "Rating List"]].mean(axis=1)

    else:
        new_frame_unlisted[code_start] = dict_unlist[code_start]["Rating Unlist"] * (
        1 + (unlist_df["Extent"].mean() / 100))
    if fa == 0:
        frame_new = new_frame_unlisted[["date", code_start]]
    else:
        frame_new = pd.merge(frame_new, new_frame_unlisted[["date", code_start]], on="date", how="left")

frame_new = frame_new.fillna(method="ffill").fillna(method="bfill")

go
go
go


In [377]:
new_frame_unlisted = pd.merge(dict_unlist[code_start], dict_list[code_start], on="date", how="left")

In [380]:
new_frame_unlisted

,date,Rating Unlist,APPB,CHIL,CPKI,TGIF,Rating List
0,2014-01-21,3.113438,3.651528,3.669427,4.269969,3.729997,3.438538
1,2014-01-22,3.103104,3.587409,3.740589,4.359797,3.601742,3.441535
2,2014-01-23,3.064960,3.705107,3.813913,4.250502,3.520541,3.458607
3,2014-01-24,3.093553,3.633207,3.701587,4.379995,3.609287,3.467216
4,2014-01-27,3.101921,3.707740,3.737475,4.363489,3.550828,3.464587
5,2014-01-28,3.064952,3.820088,3.735194,4.352409,3.444715,3.409513
6,2014-01-29,2.990932,3.938651,3.776603,4.301084,3.614843,3.631153
7,2014-01-30,2.991800,3.877053,3.696678,4.253639,3.608819,3.755641
8,2014-01-31,3.014056,3.849703,3.845146,4.329215,3.530060,3.745742
9,2014-02-03,2.970214,3.922853,3.919410,4.304893,3.557983,3.590056


In [379]:
#### This multiplier should be placed to list_df
new_frame_unlisted.fillna(method="ffill",inplace=True)

In [381]:
new_frame_unlisted["Rating Unlist"] = new_frame_unlisted["Rating Unlist"] * (
1 + (unlist_df["Extent"].mean() / 100))
new_frame_unlisted["Rating List"] = new_frame_unlisted["Rating List"] * (1 + (unlist_df["Extent"].mean() / 100))
new_frame_unlisted[code_start] = new_frame_unlisted[["Rating Unlist", "Rating List"]].mean(axis=1)

In [382]:
new_frame_unlisted

,date,Rating Unlist,APPB,CHIL,CPKI,TGIF,Rating List,BJRI
0,2014-01-21,3.161652,3.651528,3.669427,4.269969,3.729997,3.491786,3.326719
1,2014-01-22,3.151158,3.587409,3.740589,4.359797,3.601742,3.494830,3.322994
2,2014-01-23,3.112423,3.705107,3.813913,4.250502,3.520541,3.512166,3.312295
3,2014-01-24,3.141459,3.633207,3.701587,4.379995,3.609287,3.520908,3.331184
4,2014-01-27,3.149957,3.707740,3.737475,4.363489,3.550828,3.518238,3.334097
5,2014-01-28,3.112415,3.820088,3.735194,4.352409,3.444715,3.462311,3.287363
6,2014-01-29,3.037249,3.938651,3.776603,4.301084,3.614843,3.687384,3.362317
7,2014-01-30,3.038130,3.877053,3.696678,4.253639,3.608819,3.813800,3.425965
8,2014-01-31,3.060731,3.849703,3.845146,4.329215,3.530060,3.803748,3.432240
9,2014-02-03,3.016210,3.922853,3.919410,4.304893,3.557983,3.645650,3.330930


In [360]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/cpickle/")
art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
art_ratios = art_ratios["mv"][code]
art_ratios["date"] = pd.to_datetime(art_ratios["date"], infer_datetime_format=True)
art_ratios["close"] = (art_ratios["MV_pred"] / art_ratios["MV_pred"].iloc[0]) * 100
art_ratios["close"] = art_ratios["close"].rolling(60).mean()

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/stock/")
parent_frame = pd.read_csv(path + code + "_parent_tick_df.csv")
parent_frame["date"] = pd.to_datetime(parent_frame["date"], infer_datetime_format=True)
parent_frame = parent_frame.sort_values("date")
parent_frame["close"] =  (parent_frame["close"] / parent_frame["close"].iloc[0]) * 100
art_ratios = art_ratios.set_index("date")
parent_frame = parent_frame.set_index("date")
rat_par = input_fields[input_fields["code_or_ticker"] == code]["oi_ratio"].reset_index(drop=True).iloc[0]
art_ratios["close"] = art_ratios["close"]*(1-rat_par) + parent_frame["close"]*(rat_par).copy()

art_ratios["close_2"] = parent_frame["close"]
#art_ratios["close_2"] = art_ratios["close_2"]*(100/art_ratios["close_2"].iloc[0])
art_ratios["close"] = art_ratios["close"].fillna(art_ratios["close_2"])

In [363]:
art_ratios

,MV_pred,close,close_2
date,,,
2012-01-03,1532513.28,100.000000,100.000000
2012-01-04,1511653.85,99.089166,99.089166
2012-01-05,1548492.05,98.274209,98.274209
2012-01-06,1525795.88,98.897411,98.897411
2012-01-09,1524264.03,99.352828,99.352828
2012-01-10,1532578.41,104.650048,104.650048
2012-01-11,1560212.03,111.337488,111.337488
2012-01-12,1528565.27,112.080537,112.080537
2012-01-13,1520999.64,110.258869,110.258869


In [230]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../input_fields.csv")

input_fields = pd.read_csv(path)

not_listed = input_fields[input_fields["listed"] == "No"].reset_index(drop=True)["code_or_ticker"]
path = os.path.join(my_path, "data/stock/")

parents = input_fields[input_fields["parent"]=="Yes"].reset_index(drop=True)["code_or_ticker"]


In [187]:
code = "APPB"
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/cpickle/")
art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
art_ratios = art_ratios["mv"][code]
art_ratios["date"] = pd.to_datetime(art_ratios["date"], infer_datetime_format=True)
art_ratios["close"] = (art_ratios["MV_pred"] / art_ratios["MV_pred"].iloc[0]) * 100
art_ratios["close"] = art_ratios["close"].rolling(60).mean()

if code in list(parents):
    my_path = os.path.abspath(os.path.dirname('__file__'))
    path = os.path.join(my_path, "../data/stock/")
    parent_frame = pd.read_csv(path + code + "_parent_tick_df.csv")
    parent_frame["close"] =  (parent_frame["close"] / parent_frame["close"].iloc[0]) * 100
    parent_frame["date"] = pd.to_datetime(parent_frame["date"], infer_datetime_format=True)
    
    

    art_ratios = art_ratios.set_index("date")
    parent_frame = parent_frame.set_index("date")
    
    

    rat_par = input_fields[input_fields["code_or_ticker"] == code]["oi_ratio"].reset_index(drop=True).iloc[0]
    art_ratios["close"] = art_ratios["close"]*(1-rat_par) + parent_frame["close"]*(rat_par)

In [188]:
art_ratios

,MV_pred,close
date,,
2012-01-03,1532513.28,NaN
2012-01-04,1511653.85,NaN
2012-01-05,1548492.05,NaN
2012-01-06,1525795.88,NaN
2012-01-09,1524264.03,NaN
2012-01-10,1532578.41,NaN
2012-01-11,1560212.03,NaN
2012-01-12,1528565.27,NaN
2012-01-13,1520999.64,NaN


In [190]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/financial/")

In [191]:

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

dict_frames = load_dict(path+'data.pkl')


coy = "APPB"

df_input = dict_frames[coy, "calculations", "Original"].round(2)
df_input = df_input.fillna(df_input.mean()).round(2)
df_input = df_input.iloc[:-1, :]

vatios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                          ['Revenue growth', df_input["Year over Year"].iloc[-1],
                           df_input["Year over Year"].iloc[-2:].mean(),
                           df_input["Year over Year"].iloc[-3:].mean(),
                           df_input["Year over Year"].iloc[-5:].mean()],
                          ['EPS growth', df_input["Year over Year.3"].iloc[-1],
                           df_input["Year over Year.3"].iloc[-2:].mean(),
                           df_input["Year over Year.3"].iloc[-3:].mean(),
                           df_input["Year over Year.3"].iloc[-5:].mean()],
                          ['ROA', df_input["Return on Assets %"].iloc[-1],
                           df_input["Return on Assets %"].iloc[-2:].mean(),
                           df_input["Return on Assets %"].iloc[-3:].mean(),
                           df_input["Return on Assets %"].iloc[-5:].mean()],
                          ['Net Margin', df_input["Net Margin %"].iloc[-1],
                           df_input["Net Margin %"].iloc[-2:].mean(), df_input["Net Margin %"].iloc[-3:].mean(),
                           df_input["Net Margin %"].iloc[-5:].mean()],
                          ['FCF/Sales', df_input["Free Cash Flow/Sales %"].iloc[-1],
                           df_input["Free Cash Flow/Sales %"].iloc[-2:].mean(),
                           df_input["Free Cash Flow/Sales %"].iloc[-3:].mean(),
                           df_input["Free Cash Flow/Sales %"].iloc[-5:].mean()]])


,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-4.53,-5.655,-2.27667,-5.896
2,EPS growth,0.01,-1.715,62.3667,18.852
3,ROA,-16.09,-5.95,-2.52333,-0.63
4,Net Margin,-54.32,-19.43,-7.80333,-1.304
5,FCF/Sales,8.78,13.39,15.3367,16.562


In [198]:
dict_frames[coy, "calculations", "Original"].round(2)

,year,Revenue USD Mil,Gross Margin %,Operating Income USD Mil,Operating Margin %,Net Income USD Mil,Earnings Per Share USD,Dividends USD,Payout Ratio % *,Shares Mil,...,Debt/Equity,Days Sales Outstanding,Days Inventory,Payables Period,Cash Conversion Cycle,Receivables Turnover,Inventory Turnover,Fixed Assets Turnover,Asset Turnover,adj_close
0,2009,1414.0,31.7,277.0,19.6,31.0,0.55,0.01,0.01,17.0,...,59.11,28.73,4.38,15.24,17.87,12.70,83.33,1.77,0.44,19.06
1,2010,1317.0,31.4,241.0,18.3,-3.0,-1.74,0.01,0.01,17.0,...,48.49,27.85,4.66,13.05,19.46,13.10,78.39,1.93,0.45,38.74
2,2011,1055.0,36.8,220.0,20.8,75.0,3.89,0.01,0.01,18.0,...,13.96,36.40,6.23,16.88,25.75,10.03,58.59,1.98,0.39,33.12
3,2012,835.0,45.2,202.0,24.2,128.0,6.63,0.01,0.01,19.0,...,4.46,53.36,0.01,23.82,0.01,6.84,0.01,2.17,0.33,52.57
4,2013,627.0,56.8,201.0,32.0,72.0,3.70,3.00,60.30,19.0,...,4.33,79.34,0.01,47.72,0.01,4.60,0.01,2.21,0.26,68.32
5,2014,642.0,56.4,204.0,31.7,36.0,1.90,3.13,75.00,19.0,...,5.16,84.67,0.01,53.35,0.01,4.31,0.01,2.49,0.26,87.88
6,2015,670.0,57.9,223.0,33.2,105.0,5.52,3.54,116.70,19.0,...,5.26,79.70,0.01,62.62,0.01,4.58,0.01,2.90,0.28,74.53
7,2016,625.0,59.3,211.0,33.8,98.0,5.33,3.73,66.80,18.0,...,5.53,81.96,0.01,75.65,0.01,4.45,0.01,2.94,0.27,70.85
8,2017,596.0,55.9,158.0,26.4,-331.0,-18.28,3.88,0.01,18.0,...,0.01,89.21,0.01,73.18,0.01,4.09,0.01,2.95,0.30,50.73
9,2018,596.0,55.9,158.0,26.4,-331.0,-18.28,3.88,0.01,18.0,...,0.01,89.21,0.01,73.18,0.01,4.09,0.01,2.95,0.30,74.09


In [337]:
coy ="APPB"
my_path = os.path.abspath(os.path.dirname('__file__'))

path = os.path.join(my_path, "../data/cpickle/")

art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
art_ratios = art_ratios["ratios"][coy]

df_input = art_ratios.round(2)

df_input["EPS growth_pred"].iloc[:-2].mean()/df_input["Revenue growth_pred"].iloc[:-2].mean()

df_input["EPS growth_pred"].iloc[:-2].mean()/df_input["ROA_pred"].iloc[:-2].mean()

rev_rat = df_input["EPS growth_pred"].iloc[:-2].mean()/df_input["Revenue growth_pred"].iloc[:-2].mean()
roa_rat = df_input["EPS growth_pred"].iloc[:-2].mean()/df_input["ROA_pred"].iloc[:-2].mean()

df_input.loc[2017:2018,"EPS growth_pred"] = df_input.loc[2017:2018,"Revenue growth_pred"]*rev_rat*(1/2) + df_input.loc[2017:2018,"ROA_pred"]*roa_rat*(1/2)

df_input.loc[2017:2018,"EPS growth_pred"] = df_input.loc[2017:2018,"EPS growth_pred"].round(2)

ratios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                          ['Revenue growth', df_input["Revenue growth_pred"].iloc[-1],
                           df_input["Revenue growth_pred"].iloc[-2:].mean(),
                           df_input["Revenue growth_pred"].iloc[-3:].mean(),
                           df_input["Revenue growth_pred"].iloc[-5]],
                          ['EPS growth', df_input["EPS growth_pred"].iloc[-1],
                           df_input["EPS growth_pred"].iloc[-2:].mean(),
                           df_input["EPS growth_pred"].iloc[-3:].mean(),
                           df_input["EPS growth_pred"].iloc[-5:].mean()],
                          ['ROA', df_input["ROA_pred"].iloc[-1], df_input["ROA_pred"].iloc[-2:].mean(),
                           df_input["ROA_pred"].iloc[-3:].mean(), df_input["ROA_pred"].iloc[-5:].mean()],
                          ['Net Margin', df_input["Net Margin_pred"].iloc[-1],
                           df_input["Net Margin_pred"].iloc[-2:].mean(),
                           df_input["Net Margin_pred"].iloc[-3:].mean(),
                           df_input["Net Margin_pred"].iloc[-5:].mean()],
                          ['FCF/Sales', df_input["FCF/Sales_pred"].iloc[-1],
                           df_input["FCF/Sales_pred"].iloc[-2:].mean(),
                           df_input["FCF/Sales_pred"].iloc[-3:].mean(),
                           df_input["FCF/Sales_pred"].iloc[-5:].mean()]])

if coy in list(parents):
    
    def load_dict(filename_):
        with open(filename_, 'rb') as f:
            ret_di = pickle.load(f)
        return ret_di
    
    path = os.path.join(my_path, "../data/financial/")

    dict_frames = load_dict(path+'data.pkl')

    coy = "APPB"

    df_input = dict_frames[coy, "calculations", "Original"].round(2)
    df_input = df_input.fillna(df_input.mean()).round(2)
    df_input = df_input.iloc[:-1, :]

    vatios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                              ['Revenue growth', df_input["Year over Year"].iloc[-1],
                               df_input["Year over Year"].iloc[-2:].mean(),
                               df_input["Year over Year"].iloc[-3:].mean(),
                               df_input["Year over Year"].iloc[-5:].mean()],
                              ['EPS growth', df_input["Year over Year.3"].iloc[-1],
                               df_input["Year over Year.3"].iloc[-2:].mean(),
                               df_input["Year over Year.3"].iloc[-3:].mean(),
                               df_input["Year over Year.3"].iloc[-5:].mean()],
                              ['ROA', df_input["Return on Assets %"].iloc[-1],
                               df_input["Return on Assets %"].iloc[-2:].mean(),
                               df_input["Return on Assets %"].iloc[-3:].mean(),
                               df_input["Return on Assets %"].iloc[-5:].mean()],
                              ['Net Margin', df_input["Net Margin %"].iloc[-1],
                               df_input["Net Margin %"].iloc[-2:].mean(), df_input["Net Margin %"].iloc[-3:].mean(),
                               df_input["Net Margin %"].iloc[-5:].mean()],
                              ['FCF/Sales', df_input["Free Cash Flow/Sales %"].iloc[-1],
                               df_input["Free Cash Flow/Sales %"].iloc[-2:].mean(),
                               df_input["Free Cash Flow/Sales %"].iloc[-3:].mean(),
                               df_input["Free Cash Flow/Sales %"].iloc[-5:].mean()]])
    

    new = (vatios_df.iloc[1:,1:]*.80).round(2) + (ratios_df.iloc[1:,1:]*.20).round(2)
    ratios_df.iloc[1:,1:] = new
    ratios_df.iloc[1:,1:] = ratios_df.iloc[1:,1:].applymap(lambda x : round(x,2))

In [343]:
ratios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-5.99,-6.47,-3.11,-3.71
2,EPS growth,-1.9,-2.66,49.48,15.66
3,ROA,-10.54,-2.44,0.03,1.48
4,Net Margin,-43.22,-15.38,-6.07,-0.77
5,FCF/Sales,8.36,12.03,13.65,14.56


In [342]:
(vatios_df.iloc[1:,1:]*.80).round(2)

,1,2,3,4
1,-3.624,-4.524,-1.82133,-4.7168
2,0.008,-1.372,49.8933,15.0816
3,-12.872,-4.76,-2.01867,-0.504
4,-43.456,-15.544,-6.24267,-1.0432
5,7.024,10.712,12.2693,13.2496


In [333]:
if coy in parents:
    print("ff")

In [329]:
parents

0    APPB
1    CHIL
Name: code_or_ticker, dtype: object

In [328]:
new

NameError: name 'new' is not defined

In [324]:
ratios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-11.84,-9.745,-6.46333,5.04
2,EPS growth,-9.54,-6.43,-2.08333,2.88
3,ROA,11.65,11.62,10.23,9.914
4,Net Margin,1.2,0.835,0.873333,1.354
5,FCF/Sales,6.68,6.61,6.88667,6.546


In [311]:
dram = dram.reset_index()

,0,1,2,3,4
0,,,,,
index,Revenue growth,EPS growth,ROA,Net Margin,FCF/Sales
Yr 1,0.752,5.748,7.474,3.712,4.152
Yr 2,2.807,6.01,7.516,3.739,3.898
Yr 3,4.21133,25.3407,7.326,3.87067,4.28933
Yr5,7.2736,16.0064,6.4196,3.4404,2.8372


In [294]:
ratios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-11.84,-9.745,-6.46333,5.04
2,EPS growth,-9.54,-6.43,-2.08333,2.88
3,ROA,11.65,11.62,10.23,9.914
4,Net Margin,1.2,0.835,0.873333,1.354
5,FCF/Sales,6.68,6.61,6.88667,6.546


In [292]:
b

,1,2,3,4
0,-2.368,-1.949,-1.29267,1.008
1,-1.908,-1.286,-0.416667,0.576
2,2.33,2.324,2.046,1.9828
3,0.24,0.167,0.174667,0.2708
4,1.336,1.322,1.37733,1.3092


In [290]:
ratios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,5.75,6.01,25.34,16.01
2,EPS growth,7.47,7.52,7.33,6.42
3,ROA,3.71,3.74,3.87,3.44
4,Net Margin,4.15,3.9,4.29,2.84
5,FCF/Sales,NaN,NaN,NaN,NaN


In [240]:
ratios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-11.84,-9.745,-6.46333,5.04
2,EPS growth,-9.54,-6.43,-2.08333,2.88
3,ROA,11.65,11.62,10.23,9.914
4,Net Margin,1.2,0.835,0.873333,1.354
5,FCF/Sales,6.68,6.61,6.88667,6.546


In [237]:
ratios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-11.84,-9.745,-6.46333,5.04
2,EPS growth,-9.54,-6.43,-2.08333,2.88
3,ROA,11.65,11.62,10.23,9.914
4,Net Margin,1.2,0.835,0.873333,1.354
5,FCF/Sales,6.68,6.61,6.88667,6.546


In [234]:
ratios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-11.84,-9.745,-6.46333,5.04
2,EPS growth,-9.54,-6.43,-2.08333,2.88
3,ROA,11.65,11.62,10.23,9.914
4,Net Margin,1.2,0.835,0.873333,1.354
5,FCF/Sales,6.68,6.61,6.88667,6.546


In [232]:
vatios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,3.9,5.945,6.88,7.832
2,EPS growth,9.57,9.12,32.1967,19.288
3,ROA,6.43,6.49,6.6,5.546
4,Net Margin,4.34,4.465,4.62,3.962
5,FCF/Sales,3.52,3.22,3.64,1.91


In [174]:
df_input

,MV_pred,Revenue growth_pred,EPS growth_pred,ROA_pred,Interest Coverage_pred,Net Margin_pred,FCF/Sales_pred
year,,,,,,,
2012,3557206.46,12.83,24.120000,3.84,24.11,2.38,5.60
2013,4050925.22,12.66,-0.110000,2.41,37.79,1.71,5.66
2014,4439511.04,15.44,4.480000,3.07,13.42,3.30,4.92
2015,4176489.85,19.46,9.710000,2.16,20.73,2.98,7.11
2016,4100362.50,13.98,7.710000,-1.17,20.79,2.83,6.63
2017,3616875.88,8.42,-0.963454,-1.60,22.30,6.11,-0.19
2018,3679231.18,5.62,3.426790,0.76,23.46,7.53,-0.57


In [149]:
df_input["EPS growth_pred"].iloc[:-2].mean()/df_input["Revenue growth_pred"].iloc[:-2].mean()

df_input["EPS growth_pred"].iloc[:-2].mean()/df_input["ROA_pred"].iloc[:-2].mean()

rev_rat = df_input["EPS growth_pred"].iloc[:-2].mean()/df_input["Revenue growth_pred"].iloc[:-2].mean()
roa_rat = df_input["EPS growth_pred"].iloc[:-2].mean()/df_input["ROA_pred"].iloc[:-2].mean()

df_input.loc[2017:2018,"EPS growth_pred"] = df_input.loc[2017:2018,"Revenue growth_pred"]*rev_rat*(1/2) + df_input.loc[2017:2018,"ROA_pred"]*roa_rat*(1/2)

df_input

0.5215018193847172

In [229]:
ratios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-11.84,-9.745,-6.46333,5.04
2,EPS growth,-9.54,-6.43,-2.08333,2.88
3,ROA,11.65,11.62,10.23,9.914
4,Net Margin,1.2,0.835,0.873333,1.354
5,FCF/Sales,6.68,6.61,6.88667,6.546


In [228]:
vatios_df

,0,1,2,3,4
0,,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,3.9,5.945,6.88,7.832
2,EPS growth,9.57,9.12,32.1967,19.288
3,ROA,6.43,6.49,6.6,5.546
4,Net Margin,4.34,4.465,4.62,3.962
5,FCF/Sales,3.52,3.22,3.64,1.91


In [121]:
c = "BJRI"

yelp= pd.read_csv(path_in + "all_yelps_rates_" + c +".csv")

yelp["all"].iloc[-1]

yelp["all"].iloc[1]

import numpy as np


In [122]:
np.sqrt(np.abs(np.log(np.abs(yelp["all"].iloc[-1] - yelp["all"].iloc[1]))))**3.5

9.536522207273176

In [123]:
9/(np.sqrt(np.abs(np.log(np.abs(yelp["all"].iloc[-1] - yelp["all"].iloc[1]))))**3.5)

0.9437402655169209

In [112]:
c = "TGIF"

yelp= pd.read_csv(path_in + "all_yelps_rates_" + c +".csv")

yelp["all"].iloc[-1]

yelp["all"].iloc[1]

import numpy as np
np.sqrt(np.sqrt(np.abs(np.log(np.abs(yelp["all"].iloc[-1] - yelp["all"].iloc[1])))))**3

1.117695013871377

In [113]:
yelp["all"].iloc[-1]

2.136423412127169

In [114]:
yelp["all"].iloc[1]

2.3894520166626805

In [111]:
9/np.sqrt(np.sqrt(np.abs(np.log(np.abs(yelp["all"].iloc[-1] - yelp["all"].iloc[1])))))**1.4

8.05228607831627

In [ ]:
    yelp["new"] = np.sqrt(np.abs(np.log(np.abs(yelp["all"].iloc[-1] - yelp["all"].iloc[1]))))*4.7/10000

    yelp["new"] = (yelp["all"].iloc[-1]/yelp["new"].iloc[-1])*yelp["new"]


In [ ]:
yelp["new"] = yelp["all"]**9/10000

yelp["new"] = (yelp["all"].iloc[-1]/yelp["new"].iloc[-1])*yelp["new"]

In [10]:
import os
import _pickle as pickle
import pandas as pd

In [11]:
c = "APPB"

In [23]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../input_fields.csv")
path_in = os.path.join(my_path, "../data/ratings/")

input_fields = pd.read_csv(path)

code = input_fields["code_or_ticker"]

#df_tick_1= pd.read_csv(path_in +c + "_stock_rate.csv")

path_sto = os.path.join(my_path, "../data/stock/")
path_out = os.path.join(my_path, "../data/ratings/")
for c in code:
    df_tick = pd.read_csv(path_sto + c + "_tick_df.csv")

    ## This is normalised, but I don't see why I shouldn't
    df_tick = df_tick.fillna(method="bfill").fillna(method="ffill")

    df_tick["date"] = pd.to_datetime(df_tick["date"], infer_datetime_format=True)

    df_tick = df_tick[["close", "date"]]

    df_tick = df_tick[df_tick["date"]>"2012-01-01"]

    glassdoor_m= pd.read_csv(path_in +c + "_gdoor_mgmt_rate.csv")

    multiplier = (glassdoor_m["trace_mse"].tail(1)).values[0]/df_tick["close"].tail(1).values[0]

    df_tick["close"] = df_tick["close"] * multiplier

    df_tick.to_csv(path_out + c +"_stock_rate.csv")

In [29]:
c = "APPB"
df_tick = pd.read_csv(path_out + c +"_stock_rate.csv")

In [ ]:
    yelp["new"] = yelp["all"]**9/10000

    yelp["new"] = (yelp["all"].iloc[-1]/yelp["new"].iloc[-1])*yelp["new"]

In [30]:
df_tick.head()

,Unnamed: 0,close,date
0,0,2.272411,2012-01-03
1,1,2.272411,2012-01-04
2,2,2.272411,2012-01-05
3,3,2.272411,2012-01-06
4,4,2.272411,2012-01-09


In [ ]:
glassdoor = glassdoor[glassdoor["Review Date"]<=df_tick["date"].max()]

glassdoor.to_csv(path_out + cod +"_gdoor_employee_rate.csv")
glassdoor_m.to_csv(path_out + cod +"_gdoor_mgmt_rate.csv")

multiplier = (glassdoor_m["trace_mse"].tail(1)).values[0]/df_tick["close"].tail(1).values[0]

df_tick["close"] = df_tick["close"] * multiplier


df_tick.to_csv(path_out + cod +"_stock_rate.csv")

In [15]:
df_tick_1.head()

,Unnamed: 0,date,open,high,low,close,volume
0,1009,2012-01-03,43.03,43.60,41.65,2.449618,188757
1,1010,2012-01-04,41.57,41.67,41.05,2.427306,144290
2,1011,2012-01-05,41.00,41.39,40.28,2.407342,184592
3,1012,2012-01-06,41.09,41.95,40.37,2.422608,182481
4,1013,2012-01-09,41.49,41.96,41.02,2.433764,131480


In [16]:
df_tick.head()

,close,date
0,98.624485,2012-01-03
1,98.624485,2012-01-04
2,98.624485,2012-01-05
3,98.624485,2012-01-06
4,98.624485,2012-01-09


In [2]:
my_path = os.path.abspath(os.path.dirname('__file__'))

path = os.path.join(my_path, "../data/cpickle/")
first_dict = pickle.load(open(path + "first_page.p", "rb"))

In [3]:
first_dict

{'bench_code_drop_down_options': [{'label': "Applebee's ", 'value': 'APPB'},
  {'label': "Chili's", 'value': 'CHIL'},
  {'label': 'CPK', 'value': 'CPKI'},
  {'label': 'TGIF', 'value': 'TGIF'},
  {'label': 'TCF', 'value': 'CAKE'},
  {'label': 'Red Robin', 'value': 'RRGB'}],
 'c_metrics_df_output':                0    1    2    3    4    5
 0           Year  '13  '14  '15  '16  '17
 1       Solvency    ↓    ↑    ↑    ↑    ↑
 2     Efficiency    ↑    ↑    ↑    ↓    ↓
 3  Profitability    ↓    ↑    ↑    ↓    ↓
 4      Liquidity    ↑    ↑    ↑    ↓    ↑,
 'df_perf_summary_output':               one    two  three   four  five  seven  eight  nine   ten
 0                   Yr 1   Yr 2   Yr 3   Yr5   Yr 1   Yr 2  Yr 3   Yr5
 1  Revenue growth  -0.83   3.76   4.59  4.57   5.69   6.84  7.48  8.19
 2      EPS growth    -20   1.52    6.8  6.34 -63.33 -27.33   7.9  4.71
 3             ROA  10.65  10.84  10.45  10.1   4.93   5.74   6.1  5.25
 4      Net Margin   5.76   5.94   5.81  5.73    3.3   3.9

In [5]:
my_path = os.path.abspath(os.path.dirname('__file__'))

path = os.path.join(my_path, "../data/cpickle/")
first_dict_1 = pickle.load(open(path + "first_page_1.p", "rb"))

In [6]:
first_dict_1

{'bench_code_drop_down_options': [{'label': "Applebee's ", 'value': 'APPB'},
  {'label': "Chili's", 'value': 'CHIL'},
  {'label': 'CPK', 'value': 'CPKI'},
  {'label': 'TGIF', 'value': 'TGIF'},
  {'label': 'TCF', 'value': 'CAKE'},
  {'label': 'Red Robin', 'value': 'RRGB'}],
 'c_metrics_df_output':                0    1    2    3    4    5
 0           Year  '13  '14  '15  '16  '17
 1       Solvency    ↓    ↑    ↑    ↑    ↑
 2     Efficiency    ↑    ↓    ↓    ↓    ↓
 3  Profitability    ↓    ↑    ↑    ↓    ↓
 4      Liquidity    ↑    ↑    ↑    ↑    ↑,
 'df_perf_summary_output':               one    two  three   four   five  seven  eight   nine    ten
 0                   Yr 1   Yr 2   Yr 3    Yr5   Yr 1   Yr 2   Yr 3    Yr5
 1  Revenue growth  -5.19  -2.93   0.83    2.8  -1.69    1.1    3.4   5.61
 2      EPS growth  82.09  48.82  40.23  26.27  98.18  53.88  38.81  45.53
 3             ROA  11.99  11.99  11.67   10.7   6.43   6.43   6.47   6.12
 4      Net Margin   6.96   6.96   6.68   6

In [22]:
import pandas as pd
import _pickle as pickle
import datetime
from six.moves import cPickle as pickle  # for performance
import os

# After you are done editing here run app_processing and first_page_execution

##  If you edit this one also edit frames in processing
#### NB Frames is inextricably linked with app_processin.py, you have to run it for any of this to take effect.
# Also linked to first_page_execution
# Try and Except because it is linked to two files as inputs

my_path = os.path.abspath(os.path.dirname('__file__'))

path = os.path.join(my_path, "../input_fields.csv")


input_fields = pd.read_csv(path)

not_listed = input_fields[input_fields["listed"] == "No"].reset_index(drop=True)["code_or_ticker"]

s = ["'" + str(int(str(datetime.datetime.now().year)[-2:]) - 4 + i) for i in range(5)]



s_metrics_df = pd.DataFrame([["Type", "E", "C", "S", "M", "A", "BA"],
                             ["Sentiment", 3, 1, 1, 1, 1, 1],])

c_metrics_df = pd.DataFrame([["Year", s[0], s[1], s[2], s[3], s[4]],
                             ["Solvency", 1, 1, 1, 1, 1],
                             ["Efficiency", 2, 1, 1, 1, 1],
                             ["Profitability", 3, 1, 1, 1, 1],
                             ["Liquidity", 4, 1, 1, 1, 1]])


# Function To Import Dictionary and Open IT.
def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pd.read_pickle(f)
    return ret_di


import os

my_path = os.path.abspath(os.path.dirname('__file__'))


path = os.path.join(my_path, "../data/financial/")


# And the specification of this table
dict_frames = load_dict(path + 'data.pkl')  # Much rather use this one


def fin_met(coy, bench):
    if coy in list(not_listed):

        my_path = os.path.abspath(os.path.dirname('__file__'))
        try:
            path = os.path.join(my_path, "data/cpickle/")
        except:
            path = os.path.join(my_path, "../data/cpickle/")

        art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
        art_ratios = art_ratios["ratios"][coy]

        df_input = art_ratios.round(2)

        ratios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                  ['Revenue growth', df_input["Revenue growth_pred"].iloc[-1],
                                   df_input["Revenue growth_pred"].iloc[-2:].mean(),
                                   df_input["Revenue growth_pred"].iloc[-3:].mean(),
                                   df_input["Revenue growth_pred"].iloc[-5]],
                                  ['EPS growth', df_input["EPS growth_pred"].iloc[-1],
                                   df_input["EPS growth_pred"].iloc[-2:].mean(),
                                   df_input["EPS growth_pred"].iloc[-3:].mean(),
                                   df_input["EPS growth_pred"].iloc[-5:].mean()],
                                  ['ROA', df_input["ROA_pred"].iloc[-1], df_input["ROA_pred"].iloc[-2:].mean(),
                                   df_input["ROA_pred"].iloc[-3:].mean(), df_input["ROA_pred"].iloc[-5:].mean()],
                                  ['Net Margin', df_input["Net Margin_pred"].iloc[-1],
                                   df_input["Net Margin_pred"].iloc[-2:].mean(),
                                   df_input["Net Margin_pred"].iloc[-3:].mean(),
                                   df_input["Net Margin_pred"].iloc[-5:].mean()],
                                  ['FCF/Sales', df_input["FCF/Sales_pred"].iloc[-1],
                                   df_input["FCF/Sales_pred"].iloc[-2:].mean(),
                                   df_input["FCF/Sales_pred"].iloc[-3:].mean(),
                                   df_input["FCF/Sales_pred"].iloc[-5:].mean()]])


    else:
        df_input = dict_frames[coy, "calculations", "Original"].round(2)
        df_input = df_input.fillna(df_input.mean()).round(2)

        ratios_df = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                  ['Revenue growth', df_input["Year over Year"].iloc[-1],
                                   df_input["Year over Year"].iloc[-2:].mean(),
                                   df_input["Year over Year"].iloc[-3:].mean(),
                                   df_input["Year over Year"].iloc[-5:].mean()],
                                  ['EPS growth', df_input["Year over Year.3"].iloc[-1],
                                   df_input["Year over Year.3"].iloc[-2:].mean(),
                                   df_input["Year over Year.3"].iloc[-3:].mean(),
                                   df_input["Year over Year.3"].iloc[-5:].mean()],
                                  ['ROA', df_input["Return on Assets %"].iloc[-1],
                                   df_input["Return on Assets %"].iloc[-2:].mean(),
                                   df_input["Return on Assets %"].iloc[-3:].mean(),
                                   df_input["Return on Assets %"].iloc[-5:].mean()],
                                  ['Net Margin', df_input["Net Margin %"].iloc[-1],
                                   df_input["Net Margin %"].iloc[-2:].mean(), df_input["Net Margin %"].iloc[-3:].mean(),
                                   df_input["Net Margin %"].iloc[-5:].mean()],
                                  ['FCF/Sales', df_input["Free Cash Flow/Sales %"].iloc[-1],
                                   df_input["Free Cash Flow/Sales %"].iloc[-2:].mean(),
                                   df_input["Free Cash Flow/Sales %"].iloc[-3:].mean(),
                                   df_input["Free Cash Flow/Sales %"].iloc[-5:].mean()]])

    if bench in list(not_listed):

        my_path = os.path.abspath(os.path.dirname('__file__'))

        path = os.path.join(my_path, "../data/cpickle/")

        art_ratios = pickle.load(open(path + "art_ratios.p", "rb"))
        art_ratios = art_ratios["ratios"][bench]

        df_input = art_ratios.round(2)

        ratios_df_ben = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                      ['Revenue growth', df_input["Revenue growth_pred"].iloc[-1],
                                       df_input["Revenue growth_pred"].iloc[-2:].mean(),
                                       df_input["Revenue growth_pred"].iloc[-3:].mean(),
                                       df_input["Revenue growth_pred"].iloc[-5]],
                                      ['EPS growth', df_input["EPS growth_pred"].iloc[-1],
                                       df_input["EPS growth_pred"].iloc[-2:].mean(),
                                       df_input["EPS growth_pred"].iloc[-3:].mean(),
                                       df_input["EPS growth_pred"].iloc[-5:].mean()],
                                      ['ROA', df_input["ROA_pred"].iloc[-1], df_input["ROA_pred"].iloc[-2:].mean(),
                                       df_input["ROA_pred"].iloc[-3:].mean(), df_input["ROA_pred"].iloc[-5:].mean()],
                                      ['Net Margin', df_input["Net Margin_pred"].iloc[-1],
                                       df_input["Net Margin_pred"].iloc[-2:].mean(),
                                       df_input["Net Margin_pred"].iloc[-3:].mean(),
                                       df_input["Net Margin_pred"].iloc[-5:].mean()],
                                      ['FCF/Sales', df_input["FCF/Sales_pred"].iloc[-1],
                                       df_input["FCF/Sales_pred"].iloc[-2:].mean(),
                                       df_input["FCF/Sales_pred"].iloc[-3:].mean(),
                                       df_input["FCF/Sales_pred"].iloc[-5:].mean()]])

    else:
        df_input = dict_frames[bench, "calculations", "Original"].round(2)
        df_input = df_input.fillna(df_input.mean()).round(2)

        ratios_df_ben = pd.DataFrame([["", 'Yr 1', 'Yr 2', 'Yr 3', 'Yr5'],
                                      ['Revenue growth', df_input["Year over Year"].iloc[-1],
                                       df_input["Year over Year"].iloc[-2:].mean(),
                                       df_input["Year over Year"].iloc[-3:].mean(),
                                       df_input["Year over Year"].iloc[-5:].mean()],
                                      ['EPS growth', df_input["Year over Year.3"].iloc[-1],
                                       df_input["Year over Year.3"].iloc[-2:].mean(),
                                       df_input["Year over Year.3"].iloc[-3:].mean(),
                                       df_input["Year over Year.3"].iloc[-5:].mean()],
                                      ['ROA', df_input["Return on Assets %"].iloc[-1],
                                       df_input["Return on Assets %"].iloc[-2:].mean(),
                                       df_input["Return on Assets %"].iloc[-3:].mean(),
                                       df_input["Return on Assets %"].iloc[-5:].mean()],
                                      ['Net Margin', df_input["Net Margin %"].iloc[-1],
                                       df_input["Net Margin %"].iloc[-2:].mean(),
                                       df_input["Net Margin %"].iloc[-3:].mean(),
                                       df_input["Net Margin %"].iloc[-5:].mean()],
                                      ['FCF/Sales', df_input["Free Cash Flow/Sales %"].iloc[-1],
                                       df_input["Free Cash Flow/Sales %"].iloc[-2:].mean(),
                                       df_input["Free Cash Flow/Sales %"].iloc[-3:].mean(),
                                       df_input["Free Cash Flow/Sales %"].iloc[-5:].mean()]])

    ratios_df.columns = ["one", "two", "three", "four", "five"]
    ratios_df_ben.columns = ["six", "seven", "eight", "nine", "ten"]

    ratios_df_final = pd.concat((ratios_df, ratios_df_ben.iloc[:, 1:]), axis=1)
    ratios_df_final.iloc[1:, 1:] = ratios_df_final.iloc[1:, 1:].astype(float).round(2)

    return ratios_df_final


c_metrics_df_2 = c_metrics_df.set_index(0, drop=True)

new_open_string = "{BJ's}  {}."

In [23]:
fin_met("BJRI", "CAKE")

,one,two,three,four,five,seven,eight,nine,ten
0,,Yr 1,Yr 2,Yr 3,Yr5,Yr 1,Yr 2,Yr 3,Yr5
1,Revenue growth,-1.69,1.1,3.4,5.61,-5.19,-2.93,0.83,2.8
2,EPS growth,98.18,53.88,38.81,45.53,82.09,48.82,40.23,26.27
3,ROA,6.43,6.43,6.47,6.12,11.99,11.99,11.67,10.7
4,Net Margin,4.34,4.34,4.42,4.29,6.96,6.96,6.68,6.14
5,FCF/Sales,3.52,3.52,3.32,3.17,5.15,5.15,6.14,5.7


In [16]:
import os
import pandas as pd
import _pickle as pickle

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pd.read_pickle(f)
    return ret_di


my_path = os.path.abspath(os.path.dirname('__file__'))
coy = "BJRI"
path = os.path.join(my_path, "../data/financial/")
dict_frames = load_dict(path + 'data.pkl')
df_input = dict_frames[coy, "calculations", "Original"].round(2)
df_input = df_input.fillna(df_input.mean()).round(2)

In [17]:
pd.set_option("display.max_columns",None)

In [18]:
df_input = df_input.iloc[:-1,:].reset_index(drop=True)

In [19]:
df_input

,year,Revenue USD Mil,Gross Margin %,Operating Income USD Mil,Operating Margin %,Net Income USD Mil,Earnings Per Share USD,Dividends USD,Payout Ratio % *,Shares Mil,Book Value Per Share * USD,Operating Cash Flow USD Mil,Cap Spending USD Mil,Free Cash Flow USD Mil,Free Cash Flow Per Share * USD,Working Capital USD Mil,Revenue,COGS,Gross Margin,SG&A,R&D,Other,Operating Margin,Net Int Inc & Other,EBT Margin,Tax Rate %,Net Margin %,Asset Turnover (Average),Return on Assets %,Financial Leverage (Average),Return on Equity %,Return on Invested Capital %,Interest Coverage,Year over Year,3-Year Average,5-Year Average,10-Year Average,Year over Year.1,3-Year Average.1,5-Year Average.1,10-Year Average.1,Year over Year.2,3-Year Average.2,5-Year Average.2,10-Year Average.2,Year over Year.3,3-Year Average.3,5-Year Average.3,10-Year Average.3,Operating Cash Flow Growth % YOY,Free Cash Flow Growth % YOY,Cap Ex as a % of Sales,Free Cash Flow/Sales %,Free Cash Flow/Net Income,Cash & Short-Term Investments,Accounts Receivable,Inventory,Other Current Assets,Total Current Assets,Net PP&E,Intangibles,Other Long-Term Assets,Total Assets,Accounts Payable,Short-Term Debt,Taxes Payable,Accrued Liabilities,Other Short-Term Liabilities,Total Current Liabilities,Long-Term Debt,Other Long-Term Liabilities,Total Liabilities,Total Stockholders' Equity,Total Liabilities & Equity,Current Ratio,Quick Ratio,Financial Leverage,Debt/Equity,Days Sales Outstanding,Days Inventory,Payables Period,Cash Conversion Cycle,Receivables Turnover,Inventory Turnover,Fixed Assets Turnover,Asset Turnover,adj_close
0,2009,427.0,40.1,20.0,4.6,13.0,0.48,0.01,0.01,27.0,9.45,68.0,-60.0,8.0,0.31,8.00,100.0,59.89,40.11,29.77,0.01,5.73,4.62,-0.26,4.36,29.85,3.06,1.19,3.64,1.51,5.37,5.16,0.01,14.07,21.33,27.02,27.57,80.67,15.01,19.86,37.75,26.48,9.82,15.79,42.02,23.08,5.39,9.86,25.38,16.77,0.01,14.06,1.95,0.64,11.78,3.46,1.05,3.20,19.49,76.59,1.23,2.69,100.0,3.14,0.01,0.01,14.37,0.01,17.51,1.31,14.80,33.62,66.38,100.0,1.11,0.87,1.51,0.02,9.96,5.43,17.61,-2.23,36.66,67.25,1.54,1.19,18.76
1,2010,514.0,40.8,31.0,6.1,23.0,0.82,0.01,0.01,28.0,10.53,75.0,-68.0,7.0,0.24,19.00,100.0,59.21,40.79,29.07,0.01,5.62,6.09,-0.10,5.99,24.80,4.51,1.27,5.71,1.49,8.57,8.56,343.22,20.42,17.58,23.59,25.66,53.05,31.01,22.12,39.85,77.65,25.55,22.63,28.12,70.83,23.06,17.90,12.61,9.28,-20.13,13.24,1.29,0.29,12.37,2.33,1.15,3.38,19.23,76.75,1.09,2.93,100.0,4.47,0.01,0.01,10.43,0.01,14.90,0.01,18.18,33.08,66.92,100.0,1.29,0.99,1.49,0.01,8.25,5.37,18.71,-5.09,44.23,67.96,1.65,1.27,35.31
2,2011,621.0,20.4,43.0,7.0,32.0,1.08,0.01,0.01,29.0,11.50,89.0,-95.0,-6.0,0.04,13.00,100.0,79.63,20.37,7.89,0.01,5.49,6.99,0.04,7.03,27.68,5.08,1.33,6.77,1.51,10.18,10.16,304.14,20.84,18.40,21.05,25.38,44.01,58.50,27.38,23.75,36.30,45.22,26.24,25.81,31.71,40.43,21.37,13.67,18.68,0.01,15.24,-0.96,-0.19,9.04,2.90,1.19,4.20,17.33,77.79,0.93,3.95,100.0,4.73,0.01,1.68,2.76,5.60,14.77,0.01,19.01,33.79,66.21,100.0,1.17,0.81,1.51,0.01,7.22,5.43,21.36,-8.71,50.52,67.23,1.72,1.33,45.16
3,2012,708.0,19.4,41.0,5.8,31.0,1.09,0.01,0.01,29.0,13.27,87.0,-107.0,-20.0,-0.68,0.01,100.0,80.62,19.38,7.70,0.01,5.84,5.85,0.17,6.02,26.37,4.43,1.33,5.88,1.52,8.92,8.87,790.93,14.07,18.40,17.51,25.06,-4.61,28.11,25.30,54.75,-0.51,34.05,21.83,34.13,0.93,31.44,19.89,28.33,-1.74,0.01,15.05,-2.76,-0.62,5.89,3.34,1.07,4.29,14.59,80.71,0.82,3.88,100.0,4.53,0.01,1.99,2.11,5.93,14.55,0.01,19.86,34.41,65.59,100.0,1.00,0.63,1.52,0.01,8.62,3.84,15.80,-3.33,42.33,94.98,1.67,1.33,32.78
4,2013,775.0,17.5,28.0,3.6,21.0,0.73,0.01,0.01,29.0,14.03,96.0,-117.0,-22.0,-0.75,-21.00,100.0,82.48,17.52,7.62,0.01,6.32,3.58,-0.35,3.23,15.95,2.71,1.32,3.57,1.52,5.44,5.44,0.01,9.43,14.69,15.69,22.37,-33.03,-2.74,20.53,18.02,-33.07,-3.18,15.32,19.32,-33.03,-3.80,13.36,15.03,9.72,0.01,15.10,-2.78,-1.02,5.07,2.09,1.22,3.22,11.60,84.08,0.76,3.56,100.0,5.15,0.01,1.53,2.09,6.31,15.08,0.01,19.20,34.29,65.71,100.0,0.77,0.47,1.52,0.01,7.46,3.85,16.31,-4.99,48.90,94.76,1.60,1.32,30.95
5,2014,846.0,1

In [13]:
import os
import pandas as pd

my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../data/closure/")
new_frame = pd.read_csv(path + "stakeholder_metrics.csv")
new_frame = new_frame.set_index("ticker")
print(new_frame)
new_frame.columns = ['Customers', 'Management', 'Employees', 'Shareholders', "Mean"]
go = -1
for inde in list(new_frame.index):
    go = go + 1
    distances = (((new_frame[new_frame.index != inde].loc[:, :"Shareholders"] - new_frame.loc[inde,
                                                                                :"Shareholders"]) ** 2).mean(
        axis=1)) ** (1 / 2)

    frmo = distances.to_frame()

    frmo.columns = [inde]

    if go == 0:
        fula = frmo
    else:
        fula = pd.merge(fula, frmo, left_index=True, right_index=True, how="left")

        Customers  Management  Employees  Valuation Part      Mean
ticker                                                            
APPB     3.283182      2.9125   3.044000        3.744444  3.246032
BJRI     3.963182      3.3350   3.464000        3.563778  3.581490
CAKE     3.842727      3.4550   3.524000        3.518222  3.584987
CHIL     3.413636      3.0825   3.494000        3.712444  3.425645
CPKI     4.023182      3.6075   3.510000        4.134444  3.818782
RRGB     3.563636      3.0600   3.356000        3.318000  3.324409
TGIF     3.261364      2.9125   3.382000        3.594000  3.287466
Mean     3.621558      3.1950   3.396286        3.655048  3.466973


In [19]:
fula.iloc[:-1,:]

,APPB,BJRI,CAKE,CHIL,CPKI,RRGB,TGIF
ticker,,,,,,,
BJRI,0.460963,NaN,0.092985,0.311752,0.318447,0.277188,0.411914
CAKE,0.471427,0.092985,NaN,0.300623,0.330055,0.274877,0.405646
CHIL,0.249721,0.311752,0.300623,NaN,0.454288,0.222282,0.140232
CPKI,0.591583,0.318447,0.330055,0.454288,NaN,0.548005,0.585631
RRGB,0.308062,0.277188,0.274877,0.222282,0.548005,NaN,0.217932
TGIF,0.185306,0.411914,0.405646,0.140232,0.585631,0.217932,NaN


In [16]:
del fula["Mean"]

In [17]:
fula

,APPB,BJRI,CAKE,CHIL,CPKI,RRGB,TGIF
ticker,,,,,,,
BJRI,0.460963,NaN,0.092985,0.311752,0.318447,0.277188,0.411914
CAKE,0.471427,0.092985,NaN,0.300623,0.330055,0.274877,0.405646
CHIL,0.249721,0.311752,0.300623,NaN,0.454288,0.222282,0.140232
CPKI,0.591583,0.318447,0.330055,0.454288,NaN,0.548005,0.585631
RRGB,0.308062,0.277188,0.274877,0.222282,0.548005,NaN,0.217932
TGIF,0.185306,0.411914,0.405646,0.140232,0.585631,0.217932,NaN
Mean,0.285658,0.193146,0.194646,0.131082,0.378883,0.184935,0.231018


In [109]:
import os 
import pandas as pd

c = "RRGB"

input_fields = pd.read_csv(path)

codes = input_fields["code_or_ticker"]

blu =-1 
for c in codes:
    blu = blu + 1
    
    my_path = os.path.abspath(os.path.dirname('__file__'))
    path = os.path.join(my_path, "../input_fields.csv")
    path_in = os.path.join(my_path, "../data/ratings/")


    df_tick= pd.read_csv(path_in +c + "_stock_rate.csv")
    glassdoor = pd.read_csv(path_in + c + "_gdoor_employee_rate.csv")
    glassdoor_m= pd.read_csv(path_in +c + "_gdoor_mgmt_rate.csv")

    ## Alternative DF stock:

    path_in = os.path.join(my_path, "../data/stock/")
    df_tick= pd.read_csv(path_in +c + "_tick_df.csv")

    ## This is normalised, but I don't see why I shouldn't 
    df_tick = df_tick.fillna(method="bfill").fillna(method="ffill")

    df_tick = df_tick[["close","date"]]

    from scipy import signal
    glassdoor["ben_smooth"] = signal.savgol_filter(glassdoor["Final_Comp Benefits"], 199, 3)



    glassdoor["Review Date"] = pd.to_datetime(glassdoor["Review Date"], infer_datetime_format=True)
    glassdoor_m["Review Date"] = pd.to_datetime(glassdoor_m["date"], infer_datetime_format=True)
    new_door = glassdoor[["Review Date","Final_Rating","Final_Work Life Balance",
              "Final_Culture Values","Final_Career Opportunities",
              "Final_Comp Benefits","ben_smooth","Final_Senior Management"]].set_index("Review Date")

    new_door_m = glassdoor_m[["Review Date","trace_mse","trace_mwlb","trace_mcva","trace_mcop","trace_mcbe","trace_msma"]].set_index("Review Date")

    df_tick["date"] = pd.to_datetime(df_tick["date"], infer_datetime_format=True)

    df_tick = df_tick.set_index("date")

    df_tick = pd.merge(df_tick, new_door, left_index=True, right_index=True, how="outer")

    df_tick = pd.merge(df_tick, new_door_m , left_index=True, right_index=True, how="outer")

    path_in = os.path.join(my_path, "../data/ratings/")
    yelp= pd.read_csv(path_in + "all_yelps_rates_" + c +".csv")
    yelp["date"] = pd.to_datetime(yelp["date"], infer_datetime_format=True)



    df_tick = pd.merge(df_tick, yelp[["date","all","new"]].set_index("date") , left_index=True, right_index=True, how="outer")

    df_tick = df_tick.fillna(method="bfill").fillna(method="ffill")

    df_tick.head()

    if blu==0:
        df_full = df_tick
    else:
        df_full = pd.concat((df_full,df_tick),axis=0)

df_full = df_full.reset_index(drop=True)

corr_mat = df_full.corr()

from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 12)
    return pvalues


calculate_pvalues(df_full)

corr_mat

corr_mat = corr_mat[["close","Final_Work Life Balance" , "Final_Culture Values" , "Final_Career Opportunities" , "Final_Comp Benefits" , "trace_mcva", "Final_Senior Management" , "trace_msma" , "all", "new"]]  

corr_mat = corr_mat[corr_mat.index.isin(["close", "all"])]

corr_mat.columns = ["Firm Value","Work Life Balance", "Employee Culture","Career Opportunities","Compensation","Management Culture","Management Competence","Upper Management Competence","Customer Satisfaction","Customer Transformed"]

corr_mat.index =  ["Firm Value","Customer Satisfaction"]

In [110]:
corr_mat

,Firm Value,Work Life Balance,Employee Culture,Career Opportunities,Compensation,Management Culture,Management Competence,Upper Management Competence,Customer Satisfaction,Customer Transformed
Firm Value,1.000000,0.107137,0.354962,0.292417,0.327986,0.300595,0.256753,0.153506,0.084272,0.060568
Customer Satisfaction,0.084272,0.358455,0.437011,0.403017,0.359150,0.271588,0.313038,0.262023,1.000000,0.592318


In [93]:
corr_mat =corr_mat.round(2).T.reset_index().T.reset_index()
corr_mat = corr_mat.replace("index","Correlation")
path_in = os.path.join(my_path, "../data/ratings/")
corr_mat.to_csv(path_in +"corr_mat.csv",index=False)

In [ ]:
## All meassures are above six sigma statistical significance. Customer satisfaction is highly correlated with lower level employee culture and values and employees perception of career opportunities and upward mobility in the firm.  

In [103]:
yelp["all"]

0       3.159375
1       3.158785
2       3.157717
3       3.154154
4       3.155729
5       3.156856
6       3.155022
7       3.155081
8       3.153360
9       3.154053
10      3.154047
11      3.154840
12      3.153191
13      3.152579
14      3.151899
15      3.152569
16      3.150978
17      3.152928
18      3.151958
19      3.151117
20      3.150896
21      3.148302
22      3.146566
23      3.146299
24      3.145588
25      3.145505
26      3.146814
27      3.147370
28      3.147833
29      3.146944
30      3.147496
31      3.148089
32      3.147331
33      3.146454
34      3.147950
35      3.146314
36      3.145416
37      3.142632
38      3.142427
39      3.143579
40      3.143582
41      3.143695
42      3.139578
43      3.138483
44      3.139656
45      3.140499
46      3.140808
47      3.141366
48      3.138644
49      3.142521
50      3.140720
51      3.139077
52      3.137685
53      3.135744
54      3.137273
55      3.136720
56      3.136905
57      3.135336
58      3.1340

In [105]:
yelp["new"] = yelp["all"]**9/10000

yelp["new"] = (yelp["all"].iloc[-1]/yelp["new"].iloc[-1])*yelp["new"]

In [111]:
yelp.head()

,date,sugar-land,albuquerque-3,hurst,glendale,brentwood,slidell,hillsboro,fresno,sparks,...,valley-stream,burbank,chandler,plano,tyler-2,elk-grove,long-beach,north-little-rock-2,all,new
0,2012-06-02,2.954545,2.7,3.0,2.898074,2.612903,3.242857,3.233333,3.629577,3.95,...,3.884615,3.083333,3.768097,3.318831,3.285714,3.243315,3.091477,3.8,3.159375,3.385333
1,2012-06-03,2.954545,2.7,3.0,2.898074,2.677419,3.242857,3.233333,3.629577,3.95,...,3.884615,3.083333,3.768097,3.318831,3.285714,3.243315,3.091477,3.8,3.158785,3.379646
2,2012-06-04,2.954545,2.7,3.0,2.898074,2.677419,3.242857,3.233333,3.629577,3.95,...,3.884615,3.083333,3.768097,3.318831,3.285714,3.243315,3.091477,3.8,3.157717,3.369377
3,2012-06-05,2.954545,2.7,3.0,2.907376,2.677419,3.242857,3.233333,3.629577,3.95,...,3.884615,3.083333,3.768097,3.318831,3.285714,3.243315,3.091477,3.8,3.154154,3.335314
4,2012-06-06,2.954545,2.7,3.0,2.907376,2.677419,3.242857,3.233333,3.629577,3.95,...,3.884615,3.083333,3.768097,3.318831,3.285714,3.243315,3.201705,3.8,3.155729,3.350328


In [102]:
yelp["all"]**9/10000

0       3.136251
1       3.130983
2       3.121469
3       3.089913
4       3.103822
5       3.113812
6       3.097576
7       3.098093
8       3.082915
9       3.089020
10      3.088970
11      3.095966
12      3.081428
13      3.076053
14      3.070083
15      3.075964
16      3.062023
17      3.079122
18      3.070605
19      3.063242
20      3.061303
21      3.038697
22      3.023651
23      3.021345
24      3.015205
25      3.014483
26      3.025797
27      3.030613
28      3.034625
29      3.026919
30      3.031707
31      3.036843
32      3.030272
33      3.022686
34      3.035642
35      3.021471
36      3.013719
37      2.989795
38      2.988047
39      2.997917
40      2.997941
41      2.998911
42      2.963746
43      2.954457
44      2.964415
45      2.971584
46      2.974213
47      2.978975
48      2.955820
49      2.988847
50      2.973468
51      2.959495
52      2.947709
53      2.931338
54      2.944221
55      2.939555
56      2.941120
57      2.927904
58      2.9171

In [60]:
path_in = os.path.join(my_path, "../data/ratings/")

yelp= pd.read_csv(path_in + "all_yelps_rates_" + c +".csv")
yelp["date"] = pd.to_datetime(yelp["date"], infer_datetime_format=True)

path_in = os.path.join(my_path, "../data/stock/")
    
df_tick= pd.read_csv(path_in +c + "_tick_df.csv")

## This is normalised, but I don't see why I shouldn't 
df_tick = df_tick.fillna(method="bfill").fillna(method="ffill")

df_tick = df_tick[["close","date"]]

df_tick["date"] = pd.to_datetime(df_tick["date"], infer_datetime_format=True)

df_tick = df_tick.set_index("date")
    
df_tick = pd.merge(df_tick, yelp.set_index("date") , left_index=True, right_index=True, how="outer")


In [ ]:
corr_mat = df_tick.corr()

from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 12)
    return pvalues


calculate_pvalues(df_tick)

In [62]:
pd.set_option('display.max_rows',None)

In [68]:
corr_mat[corr_mat["glendale"]>0.6]

,close,sugar-land,albuquerque-3,hurst,glendale,brentwood,slidell,hillsboro,fresno,sparks,...,torrance,valley-stream,burbank,chandler,plano,tyler-2,elk-grove,long-beach,north-little-rock-2,all
glendale,0.568939,-0.071333,0.541669,0.298744,1.000000,0.706953,-0.173205,0.501547,-0.464717,0.172242,...,-0.462877,0.016433,-0.230625,-0.255942,0.517476,0.483890,0.406656,-0.633628,-0.548821,0.428694
brentwood,0.428360,0.179292,0.606408,0.314526,0.706953,1.000000,-0.221235,0.138583,-0.328041,0.066627,...,-0.375401,-0.067836,-0.279706,-0.033589,0.131985,0.686657,0.395250,-0.435997,-0.694588,0.513008
oklahoma-city,0.517667,0.138171,0.538783,0.172585,0.720153,0.757957,-0.135837,0.364361,-0.161716,0.085167,...,-0.303654,-0.071333,-0.171103,-0.109366,0.190419,0.642452,0.544487,-0.397109,-0.533206,0.540016
miamisburg-2,0.487652,0.032199,0.388940,0.203344,0.741135,0.710199,-0.048554,0.453494,-0.618892,0.023704,...,-0.470101,-0.031003,-0.362390,-0.420130,0.382975,0.538000,0.151033,-0.753199,-0.584314,0.332098
rancho-santa-margarita-2,0.455861,0.425162,0.530865,0.389851,0.670061,0.732833,-0.283627,0.172792,-0.428914,0.107259,...,-0.582400,-0.264925,-0.432495,-0.265672,0.206443,0.678118,0.196580,-0.539454,-0.707565,0.508082
eugene-2,0.292182,0.404762,0.606685,0.592456,0.682131,0.624572,-0.385407,0.201704,-0.595516,-0.080387,...,-0.811195,-0.443885,-0.660640,-0.099582,0.175424,0.712404,0.168263,-0.652626,-0.799847,0.306847
doral,0.485901,0.260376,0.546352,0.577080,0.685015,0.607009,-0.267526,0.178681,-0.487568,0.216950,...,-0.678573,-0.402296,-0.454481,-0.000774,0.398931,0.606330,0.330283,-0.535771,-0.664146,0.460408
nanuet,0.318618,0.523488,0.762709,0.582412,0.647983,0.758112,-0.400124,0.114494,-0.482071,-0.008960,...,-0.741986,-0.492565,-0.630408,0.025832,0.085107,0.815468,0.330515,-0.607347,-0.845459,0.363501
coral-springs-2,0.576179,0.011769,0.402696,0.250043,0.699554,0.538563,-0.211780,0.243901,-0.639601,0.155014,...,-0.475547,-0.112848,-0.277380,-0.325826,0.434890,0.393019,0.140668,-0.696722,-0.427941,0.268661
peoria,0.564411,0.016738,0.315793,0.303572,0.721429,0.500882,-0.031103,0.411494,-0.730906,0.238650,...,-0.599801,-0.166104,-0.399359,-0.386422,0.549720,0.331892,0.040317,-0.736769,-0.435171,0.275278
